In [1]:
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Activation, LSTM
from keras.utils import np_utils

import numpy as np
import os

In [2]:
# sample sentence
sentence = ("if you want to build a ship, don't drum up people together to"
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence)) # id -> char ['i', 'l', 'e', 'o', 'h', ...]
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
seq_length = timesteps = 10
num_classes = len(char_set)

In [3]:
dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
  x_str = sentence[i:i + seq_length]
  y_str = sentence[i + 1:i + seq_length + 1]
  print(x_str, '->', y_str)

  x = [char_dic[c] for c in x_str] # char to index
  y = [char_dic[c] for c in y_str] # char to index

  dataX.append(x)
  dataY.append(y)

if you wan -> f you want
f you want ->  you want 
 you want  -> you want t
you want t -> ou want to
ou want to -> u want to 
u want to  ->  want to b
 want to b -> want to bu
want to bu -> ant to bui
ant to bui -> nt to buil
nt to buil -> t to build
t to build ->  to build 
 to build  -> to build a
to build a -> o build a 
o build a  ->  build a s
 build a s -> build a sh
build a sh -> uild a shi
uild a shi -> ild a ship
ild a ship -> ld a ship,
ld a ship, -> d a ship, 
d a ship,  ->  a ship, d
 a ship, d -> a ship, do
a ship, do ->  ship, don
 ship, don -> ship, don'
ship, don' -> hip, don't
hip, don't -> ip, don't 
ip, don't  -> p, don't d
p, don't d -> , don't dr
, don't dr ->  don't dru
 don't dru -> don't drum
don't drum -> on't drum 
on't drum  -> n't drum u
n't drum u -> 't drum up
't drum up -> t drum up 
t drum up  ->  drum up p
 drum up p -> drum up pe
drum up pe -> rum up peo
rum up peo -> um up peop
um up peop -> m up peopl
m up peopl ->  up people
 up people -> up people 


In [4]:
# One-hot encoding
dataX = np_utils.to_categorical(dataX, num_classes = num_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))
print(dataX.shape)

(169, 10, 25)


In [5]:
# One-hot encoding
dataY = np_utils.to_categorical(dataY, num_classes = num_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))
print(dataY.shape)

(169, 10, 25)


In [6]:
model = Sequential()
model.add(LSTM(num_classes, input_shape = (timesteps, data_dim),
               return_sequences = True))
model.add(LSTM(num_classes, return_sequences = True))
model.add(TimeDistributed(Dense(num_classes)))
model.add(Activation('softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 25)            5100      
                                                                 
 lstm_1 (LSTM)               (None, 10, 25)            5100      
                                                                 
 time_distributed (TimeDistr  (None, 10, 25)           650       
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 10, 25)            0         
                                                                 
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [9]:
model.fit(dataX, dataY, epochs = 1000)

Epoch 1/1000
6/6 [==============================] - 13s 14ms/step - loss: 3.2088 - accuracy: 0.0988
Epoch 2/1000
6/6 [==============================] - 0s 10ms/step - loss: 3.1827 - accuracy: 0.1763
Epoch 3/1000
6/6 [==============================] - 0s 10ms/step - loss: 3.1507 - accuracy: 0.1840
Epoch 4/1000
6/6 [==============================] - 0s 9ms/step - loss: 3.1085 - accuracy: 0.1846
Epoch 5/1000
6/6 [==============================] - 0s 10ms/step - loss: 3.0592 - accuracy: 0.1846
Epoch 6/1000
6/6 [==============================] - 0s 9ms/step - loss: 3.0173 - accuracy: 0.1846
Epoch 7/1000
6/6 [==============================] - 0s 9ms/step - loss: 2.9912 - accuracy: 0.1846
Epoch 8/1000
6/6 [==============================] - 0s 9ms/step - loss: 2.9726 - accuracy: 0.1846
Epoch 9/1000
6/6 [==============================] - 0s 9ms/step - loss: 2.9589 - accuracy: 0.1846
Epoch 10/1000
6/6 [==============================] - 0s 12ms/step - loss: 2.9473 - accuracy: 0.1846
Epoch 11/1000

In [10]:
predictions = model.predict(dataX, verbose = 0)

for i, prediction in enumerate(predictions):
  x_index = np.argmax(dataX[i], axis = 1)
  x_str = [char_set[j] for j in x_index]

  index = np.argmax(prediction, axis = 1)
  result = [char_set[j] for j in index]

  print(''.join(x_str), ' -> ', ''.join(result))

if you wan  ->  t you want
f you want  ->  oyou want 
 you want   ->  tou want t
you want t  ->  ou want to
ou want to  ->  n want to 
u want to   ->  mwant to b
 want to b  ->  tont to bu
want to bu  ->  ont to bui
ant to bui  ->  nd to buil
nt to buil  ->  d to build
t to build  ->  hwo build 
 to build   ->  to luild a
to build a  ->  h luild a 
o build a   ->  nluild a s
 build a s  ->  tutld a sh
build a sh  ->  utld a shi
uild a shi  ->  mld a ship
ild a ship  ->  td a ship,
ld a ship,  ->  e a ship, 
d a ship,   ->   anship, d
 a ship, d  ->  tnship, do
a ship, do  ->  nship, don
 ship, don  ->  thip, don'
ship, don'  ->  iip, don't
hip, don't  ->  ep, don't 
ip, don't   ->  t, don't d
p, don't d  ->  l don't dr
, don't dr  ->   don't dru
 don't dru  ->  ton't arum
don't drum  ->   n't arum 
on't drum   ->  n't arum u
n't drum u  ->  dt drum up
't drum up  ->  t drum up 
t drum up   ->  hwrum up p
 drum up p  ->  toum up pe
drum up pe  ->   um up peo
rum up peo  ->   m up peop
u